In [ ]:
import requests
from os.path import exists
import zipfile
import folium
import pandas as pd
import geopandas as gpd
import math
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


In [ ]:
if not exists('../data/raw/sa.zip'):
    r = requests.get('https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip',allow_redirects=True)
    open('../data/raw/sa.zip', 'wb').write(r.content)
if not exists('../data/raw/SA2_2021_AUST_GDA2020.shp'):
    with zipfile.ZipFile("../data/raw/sa.zip","r") as zip_ref:
        zip_ref.extractall(path='../data/raw')


In [ ]:
# Preoprocess SA2 geometry

SA2 = gpd.read_file("../data/raw/SA2_2021_AUST_GDA2020.shp")
SA2 = SA2[SA2['STE_NAME21'] == 'Victoria'] #select victoria data
SA2 = SA2[SA2['geometry'] != None] # two instances are removed
SA2 = SA2[['SA2_CODE21','geometry']]
SA2 = SA2.rename(columns={'SA2_CODE21':'SA2 code'})
SA2['geometry'] = SA2['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
SA2['SA2area']= SA2['geometry'].area 

SA2['SA2 code'] = SA2['SA2 code'].astype(int)



/tmp/ipykernel_8744/2259639699.py:9: UserWarning:

Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [ ]:
# Preoprocess suburb geometry

gdf = gpd.read_file('../data/raw/gda94_victoriagrid/esrishape/whole_of_dataset/victoria/VMADMIN/POSTCODE_POLYGON.shp')
gdf = gdf[['POSTCODE','geometry']]
gdf['geometry'] = gdf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
gdf['subarea']= gdf['geometry'].area 
gdf['POSTCODE'] = gdf['POSTCODE'].astype(int)

/tmp/ipykernel_8744/3855079970.py:6: UserWarning:

Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [ ]:
# Gain intersection of suburb and SA2

inter = gpd.overlay(SA2,gdf, how="intersection")
inter['geometry'] = inter['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
inter['intersectionarea'] = inter['geometry'].area


/tmp/ipykernel_8744/3457130956.py:5: UserWarning:

Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [ ]:
def convert_SA2_to_suburb(file,inter,methodology,name):
    # Taking a file that contains info about something according to SA2, and inter, the geometry information of SA2 and suburb, this function converts the file information to suburb and return it
    
    df = pd.read_csv(file)
    intergdf = gpd.GeoDataFrame(
        pd.merge(df, inter, on='SA2 code', how='inner')
    )
    
    years = list(set(df.columns)-set(['SA2 code']))
    years.sort()

    # Percentage of the area of this district compared to its SA2 district
    intergdf['ratio'] = intergdf['intersectionarea'] / intergdf[methodology] 
    
    for year in years:
        # Calculate number in that district
        intergdf[year] = intergdf[year] * intergdf['ratio'] 
 
    resultdf = intergdf.groupby(['POSTCODE']).sum()[years].reset_index() #sum the number in one suburb


    # Make sure the whole ratio is 1 overall and check for outlier
    a = intergdf.groupby(name).sum('ratio').reset_index()
    plt.figure(1)
    pd.options.plotting.backend = "plotly"
    v = a.plot(kind='scatter', x=name, y='ratio').show()

    plt.figure(2)
    barWidth = 0.4
    r1 = np.arange(len(resultdf.sum()[years]))
    r2 = [x + barWidth for x in r1]

    plt.bar(r1, df.sum()[years], color='r', width=barWidth, edgecolor='white', label='original SA2 result')
    plt.bar(r2, resultdf.sum()[years], color='b', width=barWidth, edgecolor='white', label='suburb result')
    plt.xticks([r + barWidth for r in range(len(resultdf.sum()[years]))], years)
    plt.legend()
    plt.show()
    return resultdf





There is a simple example on how it is calculated. Population is proportional to SA2 area, for it is reasonable to assumn that people are equally distributed within SA2 districts. Income is proportional to suburb area, as it is not proportional to the SA2 area, hence we sum up the income in each suburb by its percentage area within suburb.

<img src="../plots/SA2toSub.jpg" width=800 height=600 />

In [ ]:
# Calculation of intersection area income is based on its percentage on suburb.

filepath=Path('../data/curated/subincome.csv')  
resultdf=convert_SA2_to_suburb('../data/curated/predictincome.csv',inter,'subarea','POSTCODE')
resultdf.to_csv(filepath, index=False)

originaldf = pd.read_csv('../data/curated/predictincome.csv')
comparedf = gpd.GeoDataFrame(
    pd.merge(SA2, originaldf, on='SA2 code', how='inner')
)

In [ ]:
comparedf.explore()

In [ ]:
comparedf = gpd.GeoDataFrame(
    pd.merge(gdf, resultdf, on='POSTCODE', how='inner')
)

In [ ]:
comparedf.explore()

In [ ]:
# Calculation of intersection area population is based on its percentage on SA2 area.

filepath=Path('../data/curated/subpopu.csv')  
resultdf=convert_SA2_to_suburb('../data/curated/predictpopu.csv',inter,'SA2area','SA2 code')
resultdf.to_csv(filepath, index=False)
originaldf = pd.read_csv('../data/curated/predictpopu.csv')
comparedf = gpd.GeoDataFrame(
    pd.merge(SA2, originaldf, on='SA2 code', how='inner')
)

In [ ]:
comparedf.explore()

In [ ]:
comparedf = gpd.GeoDataFrame(
    pd.merge(gdf, resultdf, on='POSTCODE', how='inner')
)

In [ ]:
comparedf.explore()

After observaion of both diagnose plots, transformation of population is quiet successful, while income is not that satisfactory. 

For the ratio, population is almost perfect with only several outliers, while for income, we got less than 10 per cent of dots under 0.8. 

There might be several reasons that causes the difference. For example, not accurate geometry information, not accurate geopandas area calculation, python numerical operation loses precision. However, the biggest one among them should be inadequate data from income. We got 463 instances from income and 523 instances from population, which means that some SA2 districts dont contain information about income. It can also be observed that in the first explore map, many areas in Victoria are not covered.